In [1]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd

In [2]:
model = SentenceTransformer("all-mpnet-base-v2")

In [3]:
def calc_cosine_sim(model_name, sentence_src, sentence_target):
    model = SentenceTransformer(model_name)
    embedding_src = model.encode(sentence_src, convert_to_tensor=True)
    embedding_target = model.encode(sentence_target, convert_to_tensor=True)
    cosine_score = util.pytorch_cos_sim(embedding_src, embedding_target)
    return cosine_score

In [10]:
cosine_score = calc_cosine_sim("all-mpnet-base-v2", ["raw milk", "bean milk", "milk"], ["milk industry", "cookie"])
print(cosine_score)

tensor([[0.5636, 0.2342],
        [0.5657, 0.2196],
        [0.6394, 0.3758]])


In [7]:
df_exio2fao = pd.read_excel("./data/Exio4_vs_FCL.xlsx", sheet_name="Exio4_vs_FAO")

In [8]:
df_exio2fao.head()

,description,Exio prod code,Item,Item code,Unnamed: 4,Unnamed: 5
0,Agave fibres nes,C_Agavs,Agave fibres nes,800,NaN,NaN
1,"Alcohol, Non-Food Purposes",C_Alc_food,"Alcohol, Non-Food Purposes",632,NaN,NaN
2,"Almonds, Shelled",C_Almod,"Almonds, Shelled",231,NaN,NaN
3,Almonds,C_Almol,Almonds,221,NaN,NaN
4,"Anise, badian, fennel",C_Anisr,"Anise, badian, fennel",711,NaN,NaN


In [11]:
arr_exio_descrip = df_exio2fao["description"].values
arr_fao_descrip = df_exio2fao["Item"].values

cosine_score = calc_cosine_sim("all-mpnet-base-v2", arr_fao_descrip, arr_exio_descrip)
print(cosine_score)


tensor([[1.0000, 0.2131, 0.4257,  ..., 0.3369, 0.3369, 0.3369],
        [0.2131, 1.0000, 0.2554,  ..., 0.3740, 0.3740, 0.3740],
        [0.4257, 0.2554, 1.0000,  ..., 0.3356, 0.3356, 0.3356],
        ...,
        [0.1495, 0.1134, 0.2362,  ..., 0.2224, 0.2224, 0.2224],
        [0.1783, 0.1481, 0.1615,  ..., 0.3190, 0.3190, 0.3190],
        [0.1569, 0.1499, 0.1938,  ..., 0.3497, 0.3497, 0.3497]])


In [13]:
sorted_cs, indices = cosine_score.sort(dim=1, descending=True)

In [16]:
sorted_cs[0]

tensor([1.0000, 0.6546, 0.5861, 0.5626, 0.5504, 0.5399, 0.5174, 0.5140, 0.5135,
        0.5129, 0.5118, 0.5048, 0.5031, 0.5021, 0.4977, 0.4962, 0.4836, 0.4827,
        0.4820, 0.4798, 0.4785, 0.4784, 0.4745, 0.4744, 0.4729, 0.4659, 0.4659,
        0.4646, 0.4612, 0.4601, 0.4576, 0.4572, 0.4559, 0.4558, 0.4553, 0.4534,
        0.4526, 0.4514, 0.4493, 0.4487, 0.4472, 0.4468, 0.4452, 0.4452, 0.4431,
        0.4410, 0.4408, 0.4396, 0.4382, 0.4345, 0.4342, 0.4331, 0.4322, 0.4310,
        0.4290, 0.4283, 0.4280, 0.4274, 0.4270, 0.4257, 0.4252, 0.4247, 0.4246,
        0.4236, 0.4234, 0.4208, 0.4201, 0.4196, 0.4175, 0.4162, 0.4149, 0.4134,
        0.4104, 0.4102, 0.4096, 0.4085, 0.4085, 0.4072, 0.4066, 0.4059, 0.4052,
        0.4025, 0.4017, 0.4005, 0.3998, 0.3987, 0.3987, 0.3983, 0.3980, 0.3978,
        0.3968, 0.3964, 0.3964, 0.3961, 0.3955, 0.3946, 0.3942, 0.3941, 0.3926,
        0.3916, 0.3913, 0.3906, 0.3903, 0.3897, 0.3895, 0.3890, 0.3876, 0.3871,
        0.3861, 0.3853, 0.3852, 0.3850, 

In [17]:
result_df = pd.DataFrame()

for ix, product in enumerate(arr_fao_descrip):
    sorted_product_cs = sorted_cs[ix].cpu().numpy()
    exio_ix = indices[ix].cpu().numpy()
    result_df.loc[ix, "fao_description"] = product
    result_df.loc[ix, "exio_description"] = df_exio2fao.loc[exio_ix[0], "description"]
    result_df.loc[ix, "exio_code"] = df_exio2fao.loc[exio_ix[0], "Exio prod code"]
    result_df.loc[ix, 'cosine_score'] = float("{:.3f}".format(sorted_product_cs[0]))


In [19]:
result_df.tail()

,fao_description,exio_description,exio_code,cosine_score
451,Ducks,Duck meat,C_Meat_duck,0.562
452,Turkeys,Turkey meat,C_Meat_turk,0.725
453,Asses,Meat of asses,C_Meat_ass,0.602
454,Horses,Offals of horses,C_Offas_hors,0.711
455,Rabbits,Rabbit meat,C_Meat_rabb,0.619


In [20]:
df_exio2fao.loc[df_exio2fao["Item"]=="Ducks"]

,description,Exio prod code,Item,Item code,Unnamed: 4,Unnamed: 5
451,Poultry - Meat (live),C_Polmeat,Ducks,1068,NaN,NaN


In [22]:
df_exio2fao.loc[df_exio2fao["Exio prod code"]=="C_Meat_duck"]

,description,Exio prod code,Item,Item code,Unnamed: 4,Unnamed: 5
256,Duck meat,C_Meat_duck,Duck meat,1069,NaN,NaN


In [43]:
df_merge = result_df.merge(
    df_exio2fao, 
    left_on=["fao_description"], 
    right_on=["Item"],
    how="right",
    ).drop_duplicates()

In [38]:
len(df_merge), len(df_exio2fao), len(result_df)

(456, 456, 456)

In [44]:
df_merge.columns

Index(['fao_description', 'exio_description', 'exio_code', 'cosine_score',
       'description', 'Exio prod code', 'Item', 'Item code', 'Unnamed: 4',
       'Unnamed: 5'],
      dtype='object')

In [46]:
df_merge = df_merge[["fao_description", "exio_code", "Exio prod code", "cosine_score"]]

In [48]:
df_merge["compare"] = df_merge.apply(lambda row: row["exio_code"] == row["Exio prod code"], axis=1)

In [54]:
len(df_merge[df_merge["compare"]==True])/len(df_merge)

0.9649122807017544

In [55]:
len(df_merge)

456